# Predicting House Prices using Linear Regression

This project will analyze the performance of several linear regression models, using different features for each model and understanding how the lowest error rate can be acheived. The goal will be to understand which features and parameters can most accurately predict the price of a house.

In [69]:
import pandas as pd
import numpy as np
import datetime
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

# read the dataset into a dataframe
houses = pd.read_csv('AmesHousing.tsv', sep='\t')

# explore first few rows
houses.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [70]:
# function to handle missing data or change data/data types
def transform_features(df):
    df_copy = df.copy()
    col_len = len(df_copy)
    
    # drop columns with 25% or greater missing values
    for col in df_copy.columns:
        if (df_copy[col].isnull().sum() > round(col_len * 0.25)):
            df_copy = df_copy.drop(columns=[col], axis=1)
    
    # impute the average for numeric columns if less than 5% missing
    numeric_cols = df_copy.select_dtypes(include=['int64', 'float64'])
    for cl in numeric_cols:
        if (df_copy[cl].isnull().sum() < round(col_len * 0.05)):
            df_copy[cl] = df_copy[cl].fillna(df_copy[cl].mean())
            
    # create new features based on existing columns
    df_copy['Years Until Sold'] = df_copy['Yr Sold'] - df_copy['Year Built']
    curr_year = datetime.datetime.today().year
    df_copy['Years Since Remodel'] = curr_year - df_copy['Year Remod/Add']
    
    # impute negative values for new columns to zero
    pos_vals_sold = []
    pos_vals_remod = []
    
    for val in df_copy['Years Until Sold']:
        if val < 0:
            val = 0
        pos_vals_sold.append(val)
            
    for v in df_copy['Years Since Remodel']:
        if v < 0:
            v = 0
        pos_vals_remod.append(v)
    
    df_copy['Years Until Sold'] = pos_vals_sold
    df_copy['Years Since Remodel'] = pos_vals_remod
    
    # drop columns that are no longer needed, leak sale information, or don't help machine learning
    df_copy = df_copy.drop(columns=['Year Built', 'Year Remod/Add', 'PID', 'Order', 'Mo Sold', 'Sale Condition', 'Sale Type', 'Yr Sold'], axis=1)
    return df_copy

In [71]:
# tranform the training and test data sets and verify the data
houses_transformed = transform_features(houses)
houses_transformed.head()

,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,SalePrice,Years Until Sold,Years Since Remodel
0,20,RL,141.0,31770,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,210,62,0,0,0,0,0,215000,50,59
1,20,RH,80.0,11622,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,140,0,0,0,120,0,0,105000,49,58
2,20,RL,81.0,14267,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,393,36,0,0,0,0,12500,172000,52,61
3,20,RL,93.0,11160,Pave,Reg,Lvl,AllPub,Corner,Gtl,...,0,0,0,0,0,0,0,244000,42,51
4,60,RL,74.0,13830,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,212,34,0,0,0,0,0,189900,13,21


## Feature Selection

The transform_features function above cleaned some columns, dropped some unecessary columns, and created some new columns to use in the model. The code below will analyze the correlation of the potential feature columns with the target column to understand which features will be the most effective to use

In [72]:
# generate a correlation heatmap for target column to predict
houses_transformed.corr()['SalePrice'].sort_values()

Years Until Sold      -0.558914
Years Since Remodel   -0.532974
Enclosed Porch        -0.128787
Kitchen AbvGr         -0.119814
Overall Cond          -0.101697
MS SubClass           -0.085092
Low Qual Fin SF       -0.037660
Bsmt Half Bath        -0.035815
Misc Val              -0.015691
BsmtFin SF 2           0.005890
3Ssn Porch             0.032225
Pool Area              0.068403
Screen Porch           0.112151
Bedroom AbvGr          0.143913
Bsmt Unf SF            0.182805
Lot Area               0.266549
2nd Flr SF             0.269373
Bsmt Full Bath         0.275894
Half Bath              0.285056
Open Porch SF          0.312951
Wood Deck SF           0.327143
Lot Frontage           0.357318
BsmtFin SF 1           0.432794
Fireplaces             0.474558
TotRms AbvGrd          0.495474
Mas Vnr Area           0.505784
Garage Yr Blt          0.526965
Full Bath              0.545604
1st Flr SF             0.621676
Total Bsmt SF          0.632105
Garage Area            0.640385
Garage C

In [73]:
# understand which columns to convert to categorical data type
cts = houses_transformed.select_dtypes(include='object').columns

# view number of unique categories per categorical column
for cat in cts:
    num_unique = len(houses_transformed[cat].value_counts())
    print(cat, num_unique)

MS Zoning 7
Street 2
Lot Shape 4
Land Contour 4
Utilities 3
Lot Config 5
Land Slope 3
Neighborhood 28
Condition 1 9
Condition 2 8
Bldg Type 5
House Style 8
Roof Style 6
Roof Matl 8
Exterior 1st 16
Exterior 2nd 17
Mas Vnr Type 5
Exter Qual 4
Exter Cond 5
Foundation 6
Bsmt Qual 5
Bsmt Cond 5
Bsmt Exposure 4
BsmtFin Type 1 6
BsmtFin Type 2 6
Heating 6
Heating QC 5
Central Air 2
Electrical 5
Kitchen Qual 5
Functional 8
Garage Type 6
Garage Finish 3
Garage Qual 5
Garage Cond 5
Paved Drive 3


In [74]:
# function to select the feature columns
def select_features(dfr):
    # keep only features with > 50% correlation
    features = abs(dfr.corr()['SalePrice'])
    features = features[features > 0.5]
    features = features.drop(labels=['SalePrice'])

    # get dummy data for categorical data
    categories = dfr.select_dtypes(include='object').columns

    for c in categories:
        dfr[c] = dfr[c].astype('category')

    for ct in categories:
        dfr[ct] = pd.get_dummies(dfr[ct])
        
    # drop columns not contained in features
    for column in dfr.columns:
        if column not in features:
            dfr.drop(columns=[column], axis=1)
            
    return dfr

In [75]:
# select features and verfy the data
final_houses = select_features(houses_transformed)
final_houses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 71 columns):
MS SubClass            2930 non-null int64
MS Zoning              2930 non-null uint8
Lot Frontage           2440 non-null float64
Lot Area               2930 non-null int64
Street                 2930 non-null uint8
Lot Shape              2930 non-null uint8
Land Contour           2930 non-null uint8
Utilities              2930 non-null uint8
Lot Config             2930 non-null uint8
Land Slope             2930 non-null uint8
Neighborhood           2930 non-null uint8
Condition 1            2930 non-null uint8
Condition 2            2930 non-null uint8
Bldg Type              2930 non-null uint8
House Style            2930 non-null uint8
Overall Qual           2930 non-null int64
Overall Cond           2930 non-null int64
Roof Style             2930 non-null uint8
Roof Matl              2930 non-null uint8
Exterior 1st           2930 non-null uint8
Exterior 2nd           2930 non

In [76]:
# identify null columns from final train and test dataframes
final_houses.isnull().sum().sort_values()

MS SubClass              0
Central Air              0
Electrical               0
1st Flr SF               0
2nd Flr SF               0
Low Qual Fin SF          0
Gr Liv Area              0
Bsmt Full Bath           0
Bsmt Half Bath           0
Full Bath                0
Half Bath                0
Bedroom AbvGr            0
Kitchen AbvGr            0
Kitchen Qual             0
TotRms AbvGrd            0
Functional               0
Fireplaces               0
Garage Type              0
SalePrice                0
Misc Val                 0
Pool Area                0
Screen Porch             0
3Ssn Porch               0
Enclosed Porch           0
Heating QC               0
Open Porch SF            0
Paved Drive              0
Garage Cond              0
Garage Qual              0
Garage Area              0
                      ... 
Utilities                0
Lot Config               0
Land Slope               0
Neighborhood             0
Condition 1              0
Condition 2              0
B

In [77]:
# address the missing values for Garage Yr Built and Lot Frontage
final_houses['Lot Frontage'] = final_houses['Lot Frontage'].fillna(final_houses['Lot Frontage'].mean())

final_houses = final_houses.dropna()

In [78]:
# verify the data
final_houses.isnull().sum().value_counts()

0    71
dtype: int64

In [79]:
final_houses.columns

Index(['MS SubClass', 'MS Zoning', 'Lot Frontage', 'Lot Area', 'Street',
       'Lot Shape', 'Land Contour', 'Utilities', 'Lot Config', 'Land Slope',
       'Neighborhood', 'Condition 1', 'Condition 2', 'Bldg Type',
       'House Style', 'Overall Qual', 'Overall Cond', 'Roof Style',
       'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type',
       'Mas Vnr Area', 'Exter Qual', 'Exter Cond', 'Foundation', 'Bsmt Qual',
       'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin SF 1',
       'BsmtFin Type 2', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF',
       'Heating', 'Heating QC', 'Central Air', 'Electrical', '1st Flr SF',
       '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Bsmt Full Bath',
       'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr',
       'Kitchen AbvGr', 'Kitchen Qual', 'TotRms AbvGrd', 'Functional',
       'Fireplaces', 'Garage Type', 'Garage Yr Blt', 'Garage Finish',
       'Garage Cars', 'Garage Area', 'Garage Qual', 'Garage Cond',
  

## Train and test linear regression models

The function below accepts the transformed dataframe (with the appropriate features as columns) and performs a different type of validation, based on the value of K specified. The output of the function is the RMSE or average of RMSE values based on the type of validation performed.

In [86]:
# function to train and test a Linear Regression model
def train_and_test(df, k=0):  
    features = df.columns
    features = features.drop('SalePrice')
    
    # simple validation
    if (k == 0):
        train = df[:1460]
        test = df[1460:]
          
        # train Linear Regression model
        lr = LinearRegression()
        lr.fit(train[features], train['SalePrice'])
        predictions = lr.predict(test[features])

        # calculate root mean squared error
        rmse = np.sqrt(mean_squared_error(predictions, test['SalePrice']))
        return rmse
    
    # simple cross validation
    if (k == 1):
        # shuffle the dataframe
        df_shuffled = df.sample(frac=1).reset_index(drop=True)
        fold_one = df_shuffled[:1460]
        fold_two = df_shuffled[1460:]
        
        # train Linear Regression models
        lr_1 = LinearRegression()
        lr_1.fit(fold_one[features], fold_one['SalePrice'])
        predictions_1 = lr_1.predict(fold_two[features])
        
        lr_2 = LinearRegression()
        lr_2.fit(fold_two[features], fold_two['SalePrice'])
        predictions_2 = lr_2.predict(fold_one[features])
        
        rmses = []
        # calculate root mean squared error
        rmse_1 = np.sqrt(mean_squared_error(predictions_1, fold_two['SalePrice']))
        rmse_2 = np.sqrt(mean_squared_error(predictions_2, fold_one['SalePrice']))
        rmses.append(rmse_1)
        rmses.append(rmse_2)
        
        return np.mean(rmses)
    
    # k-fold cross validation
    if (k > 1):
        # instantiate KFold object
        kf = KFold(n_splits=k, shuffle=True)
        rmse_vals = []
        
        # return the rows for training and test sets for each split (k)
        for train_index, test_index, in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            
            # fit and train Linear Regression model
            lr = LinearRegression()
            lr.fit(train[features], train['SalePrice'])
            predictions = lr.predict(test[features])
            rmse = np.sqrt(mean_squared_error(test['SalePrice'], predictions))
            rmse_vals.append(rmse)

        return np.mean(rmse_vals)
    
    else:
        print('Please specify a K value greater than zero.')


In [87]:
# verify RMSE values
rmse_simple_val = train_and_test(final_houses)
rmse_two_fold = train_and_test(final_houses, k=1)
rmse_k_folds = train_and_test(final_houses, k=5)

print('Simple Validation:', rmse_simple_val)
print('Two Fold Validation:', rmse_two_fold)
print('K Fold Validation (K=5):', rmse_k_folds)

Simple Validation: 56566.684801581636
Two Fold Validation: 32130.49790262391
K Fold Validation (K=5): 31490.774885631203


## Conclusion

As seen in the RMSE values above, the lowest average RMSE of 31,490.77 was acheived using K Fold Validation of 5 folds. 